In [73]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

import yfinance as yf
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
from pprint import pprint

from datetime import datetime
from tqdm import tqdm

In [74]:
# S&P100 전체 종목 이름, 심볼, 섹터 가져오기
res = requests.get(url = 'https://en.wikipedia.org/wiki/S%26P_100',
                    headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) \
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36'})

df_symbol = pd.read_html(res.text)[2]
df_symbol.to_csv('../data/stock_data.csv', index=False)

df_symbol.head()

,Symbol,Name,Sector
0,AAPL,Apple Inc.,Information Technology
1,ABBV,AbbVie,Health Care
2,ABT,Abbott Laboratories,Health Care
3,ACN,Accenture,Information Technology
4,ADBE,Adobe Inc.,Information Technology


In [75]:
# 이벤트 데이터 생성
df_stock = pd.read_csv('../data/df_cond1_tidy.csv')
cond = df_stock['Symbol'].isin(df_symbol.Symbol.tolist())
df_cond1 = df_stock[cond]
df_cond1.head()

df_merged = df_cond1.merge(df_symbol, how='left', on='Symbol')
df_event = df_merged[['Date', 'Symbol', 'Name', 'Sector']]
df_event.to_csv('../data/event_data.csv', index=False)
df_event.head()

,Date,Symbol,Name,Sector
0,2025-04-09,AAPL,Apple Inc.,Information Technology
1,2024-03-15,ADBE,Adobe Inc.,Information Technology
2,2024-06-14,ADBE,Adobe Inc.,Information Technology
3,2024-12-12,ADBE,Adobe Inc.,Information Technology
4,2025-03-13,ADBE,Adobe Inc.,Information Technology


In [76]:
# 테스트를 위한 이벤트 조사
np.random.seed(252)
df_event.sample(2)

,Date,Symbol,Name,Sector
75,2024-10-24,TSLA,"Tesla, Inc.",Consumer Discretionary
61,2024-02-08,PYPL,PayPal,Financials


In [77]:
ticker = yf.Ticker("TSLA")

news_list = ticker.news
target_date = datetime(2024, 10, 24).date()

# pprint(news_list)

len(ticker.get_news(count=100))
### ---> 최근 10개까지만 가능한가?
### ---> 공식 문서에 명시되어 있지 않지만 그래 보임 ㅜㅜ

10

In [ ]:
# 다른 접근법
from unittest import result
from GoogleNews import GoogleNews
googlenews = GoogleNews(lang='en')
googlenews.get_news('TESLA')
googlenews.set_time_range(start='2024-10-24', end='2024-10-24')



pd.DataFrame(googlenews.result()).head()
# 역시 최신 기사만 불러온다. 이러면 안됨.

,title,desc,date,datetime,link,img,media,site,reporter
0,Tesla stock jumps to new 2025 closing high as ...,None,5 hours ago,2025-09-23 15:39:24.619708,https://news.google.com/read/CBMirgFBVV95cUxOQ...,https://news.google.com/api/attachments/CC8iL0...,Yahoo Finance,None,By Pras Subramanian
1,Tesla Stock Has Soared 30% This Month. Here's ...,None,25 minutes ago,2025-09-23 20:14:24.619951,https://news.google.com/read/CBMinwFBVV95cUxPN...,https://news.google.com/api/attachments/CC8iL0...,Investor's Business Daily,None,By Kit Norton
2,Better EV Stock: Rivian vs. Tesla,None,39 minutes ago,2025-09-23 20:00:24.620137,https://news.google.com/read/CBMie0FVX3lxTE5TX...,https://news.google.com/api/attachments/CC8iK0...,Yahoo Finance,None,By Chris Neiger
3,Musk's robotaxi plans for San Francisco alarme...,None,Yesterday,2025-09-22 20:39:24.620317,https://news.google.com/read/CBMi1wFBVV95cUxNV...,https://news.google.com/api/attachments/CC8iK0...,Reuters,None,By Chris Kirkham
4,Three crashes in the first day? Tesla’s robota...,None,19 hours ago,2025-09-23 01:39:24.620498,https://news.google.com/read/CBMilgFBVV95cUxOT...,https://news.google.com/api/attachments/CC8iJ0...,Ars Technica,None,None


In [95]:
# 다른 접근법 Gnews
from gnews import GNews
googlenews = GNews(
    language='en',
    country='US'
)

googlenews.start_date = (2024, 10, 23)
googlenews.end_date = (2024, 10, 24)

results = pd.DataFrame(googlenews.get_news('TESLA'))
results

# 질 불러와진다. 이대로 사용하자.

,title,description,published date,url,publisher
0,Tesla surprises with sales forecast and cost e...,Tesla surprises with sales forecast and cost e...,"Wed, 23 Oct 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMitwFBV...,"{'href': 'https://www.reuters.com', 'title': '..."
1,The fastest-growing part of Tesla’s business i...,The fastest-growing part of Tesla’s business i...,"Thu, 24 Oct 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMihwFBV...,"{'href': 'https://sherwood.news', 'title': 'Sh..."
2,Tesla adds close to $150bn in market value on ...,Tesla adds close to $150bn in market value on ...,"Thu, 24 Oct 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMifkFVX...,"{'href': 'https://www.theguardian.com', 'title..."
3,"Elon Musk Says Tesla's $25K Model Is Dead, Roa...","Elon Musk Says Tesla's $25K Model Is Dead, Roa...","Thu, 24 Oct 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMinAFBV...,"{'href': 'https://www.roadandtrack.com', 'titl..."
4,Chilling Moment Tesla Sensors Pick Up ‘Somethi...,Chilling Moment Tesla Sensors Pick Up ‘Somethi...,"Wed, 23 Oct 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMikgFBV...,"{'href': 'https://www.newsweek.com', 'title': ..."
...,...,...,...,...,...
95,Elon Musk finally admits Tesla’s HW3 might not...,Elon Musk finally admits Tesla’s HW3 might not...,"Wed, 23 Oct 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMipgFBV...,"{'href': 'https://electrek.co', 'title': 'Elec..."
96,Uber Stock Slides As Tesla Robotaxi Threat Ree...,Uber Stock Slides As Tesla Robotaxi Threat Ree...,"Thu, 24 Oct 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMinwFBV...,"{'href': 'https://www.investors.com', 'title':..."
97,Tesla’s price cuts worked – while keeping marg...,Tesla’s price cuts worked – while keeping marg...,"Thu, 24 Oct 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMinAFBV...,"{'href': 'https://www.whichev.net', 'title': '..."
98,Tesla aims for 2M+ Cybercab production annuall...,Tesla aims for 2M+ Cybercab production annuall...,"Thu, 24 Oct 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMib0FVX...,"{'href': 'https://www.teslarati.com', 'title':..."
